In [2]:
import numpy as np
import os
import tensorflow as tf
from IPython.display import clear_output, Image, display, HTML

###### Do not modify here ###### 
def strip_consts(graph_def, max_const_size=32):
    """Strip large constant values from graph_def."""
    strip_def = tf.GraphDef()
    for n0 in graph_def.node:
        n = strip_def.node.add() 
        n.MergeFrom(n0)
        if n.op == 'Const':
            tensor = n.attr['value'].tensor
            size = len(tensor.tensor_content)
            if size > max_const_size:
                tensor.tensor_content = "<stripped %d bytes>"%size
    return strip_def

def show_graph(graph_def, max_const_size=32):
    """Visualize TensorFlow graph."""
    if hasattr(graph_def, 'as_graph_def'):
        graph_def = graph_def.as_graph_def()
    strip_def = graph_def
    #strip_def = strip_consts(graph_def, max_const_size=max_const_size)
    code = """
        <script>
          function load() {{
            document.getElementById("{id}").pbtxt = {data};
          }}
        </script>
        <link rel="import" href="https://tensorboard.appspot.com/tf-graph-basic.build.html" onload=load()>
        <div style="height:600px">
          <tf-graph-basic id="{id}"></tf-graph-basic>
        </div>
    """.format(data=repr(str(strip_def)), id='graph'+str(np.random.rand()))

    iframe = """
        <iframe seamless style="width:1200px;height:620px;border:0" srcdoc="{}"></iframe>
    """.format(code.replace('"', '&quot;'))
    display(HTML(iframe))

###### Do not modify here ###### 

# to make this notebook's output stable across runs
def reset_graph(seed=42):
    tf.reset_default_graph()
    tf.set_random_seed(seed)
    np.random.seed(seed)

reset_graph()

from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("/tmp/data/")

# training on MNIST but only on digits 0 to 4
X_train1 = mnist.train.images[mnist.train.labels < 5]
y_train1 = mnist.train.labels[mnist.train.labels < 5]
X_valid1 = mnist.validation.images[mnist.validation.labels < 5]
y_valid1 = mnist.validation.labels[mnist.validation.labels < 5]
X_test1 = mnist.test.images[mnist.test.labels < 5]
y_test1 = mnist.test.labels[mnist.test.labels < 5]

###### Do not modify here ###### 



Extracting /tmp/data/train-images-idx3-ubyte.gz
Extracting /tmp/data/train-labels-idx1-ubyte.gz
Extracting /tmp/data/t10k-images-idx3-ubyte.gz
Extracting /tmp/data/t10k-labels-idx1-ubyte.gz


## Define Graph

In [97]:
def fully_connected_layer(x, dims, activate, name):
    '''
    [Function Input]
    x        "Tensor" : The input tensor
    dims     "List"   : The dimension of Input and Output
    activate "String" : The name of activate function
    name     "String" : The name of this operation
    
    [Function Output]
    The Result of Linear Function + Activate Function 
    '''
    with tf.variable_scope(name):
        W = tf.get_variable("Weights", dims, initializer=tf.contrib.layers.variance_scaling_initializer()) # Initialize Weight with He
        b = tf.get_variable("Bias", [1], initializer=tf.constant_initializer(0)) # Initialize Bias with constant
        output = tf.matmul(x, W) + b # Connect each elements for model
        
        # Define Activate Function
        if activate == 'elu':
            output = tf.nn.elu(output, name="Elu")
        elif activate == 'softmax':
            output = tf.nn.softmax(output, name="Softmax")
    return output

# Clear graph
tf.reset_default_graph()

# Define Input Entry
x = tf.placeholder(tf.float32, shape=(None, 784), name="Input_X") # Input
y = tf.placeholder(tf.int32, shape=(None,), name ="Input_Label")  # Labels

# Define Connect Layers
x1 = fully_connected_layer(x , [784, 128], "elu", "Fully_Connected_Layer_1") # 1st 128 nodes Layer
# x1 = tf.layers.dense(inputs=x, units=128, activation=tf.nn.elu, name="Fully_Connected_Layer_1")  # 1st 128 nodes Layer using dense
x2 = fully_connected_layer(x1, [128, 128], "elu", "Fully_Connected_Layer_2") # 2nd 128 nodes Layer
x3 = fully_connected_layer(x2, [128, 128], "elu", "Fully_Connected_Layer_3") # 3rd 128 nodes Layer
x4 = fully_connected_layer(x3, [128, 128], "elu", "Fully_Connected_Layer_4") # 4th 128 nodes Layer
x5 = fully_connected_layer(x4, [128, 128], "elu", "Fully_Connected_Layer_5") # 5th 128 nodes Layer
y_ = fully_connected_layer(x5, [128, 5], "softmax", "Softmax_Layer")         # 128 to 5 class with softmax

# Define Loss Function
cross_entropy = tf.reduce_mean(tf.nn.sparse_softmax_cross_entropy_with_logits(labels=y, logits=y_, name="Cross_Entropy"))

# Define Training Process
train_step = tf.train.AdamOptimizer(0.017).minimize(cross_entropy)


In [99]:
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    
    show_graph(tf.get_default_graph().as_graph_def())
    for epochs in range(1000):
        sess.run(train_step, feed_dict={x:X_train1[:1024], y:y_train1[:1024]})
        
        if epochs % 100 == 0:
            print(sess.run(cross_entropy, feed_dict={x:X_valid1[:1024], y:y_valid1[:1024]}))
    

1.63802
1.53276
1.53276
1.53276
1.53276
1.53276
1.53276
1.53276
1.53276
1.53276
